In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import pandas
import os
import numpy as np
import pickle as pk
import sklearn as sk
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [5]:
data = pandas.read_csv("./../codes/tuke-codes-representation")
data.dropna()
data = data[2:]
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

[]
0 Physical GPU, 0 Logical GPUs


2023-05-23 13:17:36.141258: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-23 13:17:36.141306: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: Anonymous
2023-05-23 13:17:36.141314: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: Anonymous
2023-05-23 13:17:36.141527: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 530.30.2
2023-05-23 13:17:36.141560: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.30.2
2023-05-23 13:17:36.141567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 530.30.2


In [6]:
typeMap = {int(data.iloc[x,0].split(".")[0]):data.iloc[x,1:].values.tolist() for x in range(len(data))}
pickle_save = "/home/anonymous/PycharmProjects/thesisPractical/pickle/java-save"
x = []
y = []

with open(f"{pickle_save}/data.pkl", "rb") as d:
    da = pandas.read_pickle(d)
    for j in da:
        for i in j.values:
            if typeMap.get(i[0], 0) and typeMap.get(i[1], 0):
                x.append([typeMap[i[0]], typeMap[i[1]]])
                y.append(True)
with open(f"{pickle_save}/false/data.pkl", "rb") as d:
    da = pandas.read_pickle(d)
    for j in da:
        for i in j.values:
            if typeMap.get(i[0],0) and typeMap.get(i[1],0):
                x.append([typeMap[i[0]], typeMap[i[1]]])
                y.append(False)

ValueError: invalid literal for int() with base 10: 'arrays10011-cpg'

In [7]:
tf.random.set_seed(12)
tf.keras.backend.clear_session()
x = np.array(x).astype("float64")
y = np.array(y).astype("int")

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=23,shuffle=True)
x_train_0 = np.array([x[0] for x in x_train])
x_train_1 = np.array([x[1] for x in x_train])
x_test_0 = np.array([x[0] for x in x_train])
x_test_1 = np.array([x[1] for x in x_train])

def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, tf.keras.backend.epsilon()))

def create_network(input_size):
    # Define the tensors for the two input images
    input_1 = Input(shape=(input_size,))
    input_2 = Input(shape=(input_size,))

    # Neural Network
    model = tf.keras.Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_size,)))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))

    # Generate the encodings (feature vectors) for the two images
    encoded_1 = model(input_1)
    encoded_2 = model(input_2)

    # Add a customized layer to compute the Euclidean distance between the encodings
    distance = Lambda(euclidean_distance)([encoded_1, encoded_2])

    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1, activation='sigmoid')(distance)

    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[input_1, input_2], outputs=prediction)

    # return the model
    return siamese_net

# Create the siamese network
siamese_net = create_network(16)

# Compile the model
siamese_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
siamese_net.summary()

NameError: name 'x' is not defined

In [22]:
history = model.fit([x_train_0,x_train_1], y_train, epochs=40)

Epoch 1/40
260/260 [==============================] - 1s 2ms/step - loss: 0.0257 - accuracy: 0.9951
Epoch 2/40
260/260 [==============================] - 1s 2ms/step - loss: 0.0075 - accuracy: 0.9993
Epoch 3/40
260/260 [==============================] - 1s 2ms/step - loss: 0.0125 - accuracy: 0.9992
Epoch 4/40
260/260 [==============================] - 1s 2ms/step - loss: 0.0133 - accuracy: 0.9996
Epoch 5/40
260/260 [==============================] - 1s 3ms/step - loss: 1.0919e-15 - accuracy: 1.0000
Epoch 6/40
260/260 [==============================] - 1s 2ms/step - loss: 1.0919e-15 - accuracy: 1.0000
Epoch 7/40
260/260 [==============================] - 1s 2ms/step - loss: 1.0919e-15 - accuracy: 1.0000
Epoch 8/40
260/260 [==============================] - 1s 2ms/step - loss: 1.0919e-15 - accuracy: 1.0000
Epoch 9/40
260/260 [==============================] - 1s 2ms/step - loss: 1.0919e-15 - accuracy: 1.0000
Epoch 10/40
260/260 [==============================] - 1s 3ms/step - loss: 1.091

In [8]:

y_pred = model.evaluate([left_x_test, right_x_test],right_y_test)


NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# Assuming that 'model' is your trained model and 'X_test', 'y_test' are your test data.

# Predict the probabilities for the test data
y_pred_prob = model.predict([left_x_test, right_x_test]).ravel()

# Predict the classes for the test data
y_pred = np.round(y_pred_prob)

# Print precision, recall, and F1-score
print(classification_report(left_y_test, y_pred))

# Calculate AUC-ROC
roc_auc = roc_auc_score(left_y_test, y_pred_prob)
print("AUC-ROC:", roc_auc)

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(left_y_test, y_pred_prob)

# Plot ROC curve
plt.figure()
lw = 2  # Line width
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming that 'model' is your trained model, and 'history' is the returned History object from model.fit
# e.g., history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20)

# Plot training & validation accuracy values
plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()